In [3]:
# Importing important libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [38]:
# Getting the whole content of the website (IMDB Barbie reviews)

url = 'https://letterboxd.com/film/barbie/reviews/by/activity/'

# Iterating through each page of the website
for page_number in range(1, 5):
    letterboxd_url = f'{url}&start={25 * (page_number - 1)}'

    letterboxd_response = requests.get(letterboxd_url)
    letterboxd_soup = BeautifulSoup(letterboxd_response.content, 'html.parser')
print(letterboxd_response)

<Response [404]>


In [34]:
letterboxd_response

<Response [404]>

In [26]:
# Defining our lists

review = []
rating = []

In [30]:
# Extracting the needed data and appending it to variables

user_review = letterboxd_soup.find_all('div', class_ = 'text show-more__control')
user_rating = letterboxd_soup.find_all('span', class_ = 'rating-other-user-rating')

print(len(user_review))
print(len(user_rating))


25
24


In [31]:
# Print the first few user reviews and ratings
for i in range(100):  # Print the first 5 elements for review and rating
    print(f"User Review {i + 1}: {user_review[i].text.strip()}")
    print(f"User Rating {i + 1}: {user_rating[i].text.strip()}")
    print("\n")  # Add a separator


User Review 1: Margot does the best with what she's given, but this film was very disappointing to me. It was marketed as a fun, quirky satire with homages to other movies. It started that way, but ended with over-dramatized speeches and an ending that clearly tried to make the audience feel something, but left everyone just feeling confused. And before you say I'm a crotchety old man, I'm a woman in my 20s, so I'm pretty sure I'm this movie's target audience. The saddest part is there were parents with their kids in the theater that were victims of the poor marketing, because this is not a kid's movie. Overall, the humor was fun on occasion and the film is beautiful to look at, but the whole concept falls apart in the second half of the film and becomes a pity party for the "strong" woman.
User Rating 1: 6/10


User Review 2: The first reason to go see it:It's good fun. It's fairytale comedy, over the top, pastiche and it works! It simply does! Margot Robie and Ryan Gosling have great

IndexError: list index out of range